In [1]:
import numpy as np
from collections import defaultdict

## 数据准备

In [2]:
item2cate = {}
with open('../../data/book_data/book_item_cate.txt', 'r') as f:
    for line in f:
        conts = line.strip().split(',')
        item2cate[conts[0]] = conts[1]
id2asin = {}
book_list = set()
with open('../../data/book_data/book_item_map.txt', 'r') as f:
    for line in f:
        conts = line.strip().split(',')
        id2asin[conts[1]] = conts[0]
        book_list.add(conts[0])
id2cate = {}
with open('../../data/book_data/book_cate_map.txt', 'r') as f:
    for line in f:
        conts = line.strip().split(',')
        id2cate[conts[-2]] = conts[-3]

In [3]:
bookinfo = defaultdict(dict)
with open('../../../meta_Books.json', 'r') as f:
    for line in f:
        r = eval(line.strip())
        asin = r['asin'] 
        if asin not in book_list:
            continue
        bookinfo[asin]['title'] = r['title']
        if len(r['category'])>0:
            bookinfo[asin]['cate'] = r['category'][-1]

## 筛选

In [73]:
exp_name = 'default_7'
info_path = './info/' + exp_name + '_info.txt'

users = set()
user2intere = defaultdict(list)
user2cate = defaultdict(dict)
user2item = defaultdict(dict)

with open(info_path, 'r') as f:
    for line in f:
        conts = line.strip().split(',')
        user = conts[0]
        intere = conts[1]
        users.add(user)
        user2intere[user].append(intere)
        user2cate[user][intere] = set()
        user2item[user][intere] = []
        for i in range(2,52):
            item = conts[i]
            if item in item2cate:
                cate = item2cate[item]
            else:
                cate = '1'
            if cate != '1':
                user2cate[user][intere].add(cate)
                user2item[user][intere].append((id2asin[item],id2cate[cate]))

In [74]:
num = len(users)
score = dict()
for u in users:
    sco = 0
    for j in range(4):
        intere1 = user2intere[u][j]
        for k in range(j+1,4):
            intere2= user2intere[u][k]
            set1 = user2cate[u][intere1]
            set2 = user2cate[u][intere2]
            sco += len(set1&set2)/(len(set1)+len(set2))
    score[u] = sco

In [75]:
candidates = sorted(score.items(),key = lambda x:x[1],reverse = False)
i=0

In [88]:
u = candidates[i][0]
for k, v in user2item[u].items():
    user2item[u][k]=sorted(v,key=lambda v:v[1])
print(i)
i += 1
user2item[u]

9


{'17': [('159448256X', ' Research &amp; Publishing Guides'),
  ('1573240648', 'Behavioral Sciences'),
  ('0692200770', 'Contemporary'),
  ('1492206776', 'Dramas & Plays'),
  ('1493761005', 'Erotica'),
  ('B00COTRYHW', 'Genre Fiction'),
  ('1455583650', 'Genre Fiction'),
  ('0761536957', 'Higher &amp; Continuing Education'),
  ('1602826706', 'Literature &amp; Fiction'),
  ('3540224947', 'Mathematics'),
  ('1617130079', 'Music'),
  ('160163059X', 'Occult & Paranormal'),
  ('1451684568', 'Performing Arts'),
  ('149293304X', 'Personal Finance'),
  ('0812981561', 'Philosophy'),
  ('0615782043', 'Poetry'),
  ('158367182X', 'Politics & Government'),
  ('1430242337', 'Programming'),
  ('1560974109', 'Publishers'),
  ('1476716455', 'Schools &amp; Teaching'),
  ('1469732459', 'Short Stories &amp; Anthologies'),
  ('0307986497', 'World')],
 '63': [('0307278824', ' Instruments &amp; Measurement'),
  ('0061977969', 'Action &amp; Adventure'),
  ('0062011847', 'Action &amp; Adventure'),
  ('193649352

## 存入信息

In [80]:
final_info = []
for k,v in user2item[u].items():
    final_info.append([(x[0], bookinfo[x[0]]['cate'], bookinfo[x[0]]['title']) for x in v])

In [81]:
import csv  
with open('./result/{}_{}.csv'.format(exp_name,u), 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['asin', 'cate', 'title'])
    for a_list in final_info:
        for asin, cate, title in a_list:
            writer.writerow([asin, cate, title])
        writer.writerow(['*'])

In [82]:
'{}_{}.csv'.format(exp_name,u)

'default_6_586878.csv'